In [6]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# feature name index and feature name are seperated by spaces in the features.txt. load it into a DataFrame.
feature_name_df = pd.read_csv('./human_activity/features.txt', sep='\s+', header=None, \
                             names = ['columns_index', 'column_name'])

# Remove feature name index, and create features name as list objects, extract only 10 samples
feature_name = feature_name_df.iloc[:, 1].values.tolist()
print('Extract only 10 from all feature name : ', feature_name[:10])

Extract only 10 from all feature name :  ['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X']


In [21]:
feature_dup_df = feature_name_df.groupby('column_name').count()
print(feature_dup_df[feature_dup_df['columns_index'] > 1].count())
feature_dup_df[feature_dup_df['columns_index'] >1].head()

columns_index    42
dtype: int64


,columns_index
column_name,
"fBodyAcc-bandsEnergy()-1,16",3
"fBodyAcc-bandsEnergy()-1,24",3
"fBodyAcc-bandsEnergy()-1,8",3
"fBodyAcc-bandsEnergy()-17,24",3
"fBodyAcc-bandsEnergy()-17,32",3


In [27]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby(
        'column_name').cumcount(), columns=['dup_cnt'])
    
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(),
                                  feature_dup_df, how='outer')
    new_feature_name_df['column_name']=new_feature_name_df[['column_name',\
        'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) if x[1] > 0 else x[0], axis=1)
    
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df
    

In [28]:
import pandas as pd

def get_human_datset():
    
    # Each data file is seperated by space, so read_csv assign a space chracter to sep
    feature_name_df = pd.read_csv('./human_activity/features.txt', sep='\s+',
                                 header=None, names=['column_index', 'column_name'])
    # Create new feature name DataFrame using get_new_feature_name_df() to correct
    # duplicate feature names
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # Convert back to list object to give feature name to column in DataFrame
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # loading training feature dataset and test feature data into DataFrame. Apply feautre_name to column name
    X_train = pd.read_csv('./human_activity/train/X_train.txt', sep='\s+', names=feature_name)
    X_test = pd.read_csv('./human_activity/train/X_test.txt', sep='\s+', names=feature_name)
    
    # loading training label and test label data into DataFrame, give column name to action
    y_train = pd.read_csv('./human_action/train/y_train.txt', sep='\s+', header=None, names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt', sep='\s+', header=None, names=['action'])
    